In [6]:
import psycopg2
conn = psycopg2.connect(host="localhost", database="covid19", user="postgres", password="lgribeiro")

In [7]:
cur = conn.cursor()

sql = "SELECT COUNT(P.IC_SEXO) FILTER (where UPPER(P.IC_SEXO) = 'F') as FEMALE,	COUNT(P.IC_SEXO) FILTER (where UPPER(P.IC_SEXO) = 'M') as MALE, 	FLOOR((date_part('year', CURRENT_DATE)-P.AA_NASCIMENTO::INTEGER)/10) || '0s' as DECADA FROM PACIENTE P WHERE P.AA_NASCIMENTO SIMILAR TO '%(0|9)%' GROUP BY DECADA;" 


In [8]:
cur.execute(sql)

rows = cur.fetchall()
    
for row in rows:
    print(row)

(3915, 4316, '00s')
(5505, 5428, '10s')
(30366, 25824, '20s')
(53306, 42775, '30s')
(39142, 33398, '40s')
(22043, 20798, '50s')
(13475, 11151, '60s')
(6667, 5793, '70s')
(3720, 2164, '80s')


In [9]:
sql1 = "with ages as (select min(date_part('year', CURRENT_DATE)- P.AA_NASCIMENTO::int)::int as min_age, max(date_part('year', CURRENT_DATE)- P.AA_NASCIMENTO::int)::int as max_age from PACIENTE P WHERE P.AA_NASCIMENTO SIMILAR TO '%(0|9)%' ), histogram as ( select width_bucket((date_part('year', CURRENT_DATE)- P.AA_NASCIMENTO::int), min_age, max_age, 9) as bucket, int4range(min(date_part('year', CURRENT_DATE)- P.AA_NASCIMENTO::int)::int, max(date_part('year', CURRENT_DATE)- P.AA_NASCIMENTO::int)::int, '[]') as range, COUNT(P.IC_SEXO) FILTER (where UPPER(P.IC_SEXO) = 'F') as FEMALE, COUNT(P.IC_SEXO) FILTER (where UPPER(P.IC_SEXO) = 'M') as MALE from paciente P, ages WHERE P.AA_NASCIMENTO SIMILAR TO '%(0|9)%' group by bucket order by bucket ) select bucket, range, FEMALE, MALE from histogram;"    



In [10]:
cur.execute(sql1)

rows = cur.fetchall()
    
for row in rows:
    print(row)

(1, NumericRange(0, 10, '[)'), 3915, 4316)
(2, NumericRange(10, 20, '[)'), 5505, 5428)
(3, NumericRange(20, 30, '[)'), 30366, 25824)
(4, NumericRange(30, 40, '[)'), 53306, 42775)
(5, NumericRange(40, 50, '[)'), 39142, 33398)
(6, NumericRange(50, 60, '[)'), 22043, 20798)
(7, NumericRange(60, 70, '[)'), 13475, 11151)
(8, NumericRange(70, 80, '[)'), 6667, 5793)
(9, NumericRange(80, 89, '[)'), 3424, 2061)
(10, NumericRange(89, 90, '[)'), 296, 103)


In [12]:
cur.close()
